# EDA 3
- This will continue from EDA 2 df_anime_export.csv
- Purpose is to use this df for content recommender

In [2]:
import numpy as np 
import os 
import pandas as pd 

In [3]:
df_anime = pd.read_csv("../data/df_anime_export.csv")

# View Resetter

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [5]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

# Replicating Cleaning done in content_recc final

In [6]:
df_anime[df_anime['synopsis_processed'].isnull()]

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
1556,1557,38475,Yuru Camp△ Movie,,"['Comedy', 'Slice of Life']",Not available,1.0,17337,3447,-1.0,-1.00,NaN
1670,1671,38476,Heya Camp△,,"['Slice of Life', 'Comedy']","Jan 6, 2020 to ?",-1.0,22331,3048,-1.0,-1.00,NaN
8819,9385,38341,Bai Dao,,[],"Jun 13, 2017",1.0,117,15025,11609.0,5.44,NaN
11127,11785,34755,Kuma no Gakkou: Patissier Jackie to Ohisama no...,,"['Comedy', 'Kids']","Aug 25, 2017",1.0,142,14479,12812.0,6.31,NaN
13570,14260,40714,Youkai Watch Jam: Youkai Gakuen Y - N to no So...,,"['Comedy', 'Demons', 'Kids', 'Supernatural', '...","Dec 27, 2019 to ?",-1.0,290,12280,14557.0,6.82,NaN
13634,14326,34794,Yukai na Animal Bus,,"['Comedy', 'Kids']","Jan 8, 2017 to Mar 26, 2017",12.0,162,14145,14570.0,5.32,NaN
13665,14357,40421,Given Movie,,"['Music', 'Slice of Life', 'Drama', 'Romance',...",2020,1.0,15424,3682,-1.0,-1.00,NaN


In [7]:
for idx, genre in enumerate(df_anime['genre']):
    df_anime.at[idx, 'genre'] = genre.replace('Martial Arts', 'Martial_Arts')
    df_anime.at[idx, 'genre'] = genre.replace('Super Power', 'Super_Power')
    df_anime.at[idx, 'genre'] = genre.replace('Slice of Life', 'Slice_of_Life')

#clean genre so it can be used in the rec system
for idx, row_genre in enumerate(df_anime['genre']):
    row_genre = row_genre.replace("'",'')
    row_genre = row_genre.replace(",",'')
    row_genre = row_genre.replace("]","")
    row_genre = row_genre.replace("[","")
    df_anime['genre'].iloc[idx] = row_genre

# Calculate the minimum and maximum values of the 'popularity' column
min_popularity = df_anime['popularity'].min()
max_popularity = df_anime['popularity'].max()

# Perform min-max scaling on the 'popularity' column
df_anime['popularity_normalized'] = (df_anime['popularity'] - min_popularity) / (max_popularity - min_popularity)
df_anime.dropna(inplace = True)
df_anime.info()

C:\Users\samue\AppData\Local\Temp\ipykernel_3728\4109322509.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anime['genre'].iloc[idx] = row_genre


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16207 entries, 0 to 16213
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             16207 non-null  int64  
 1   uid                    16207 non-null  int64  
 2   title                  16207 non-null  object 
 3   synopsis               16207 non-null  object 
 4   genre                  16207 non-null  object 
 5   aired                  16207 non-null  object 
 6   episodes               16207 non-null  float64
 7   members                16207 non-null  int64  
 8   popularity             16207 non-null  int64  
 9   ranked                 16207 non-null  float64
 10  score                  16207 non-null  float64
 11  synopsis_processed     16207 non-null  object 
 12  popularity_normalized  16207 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 1.7+ MB


# Begin trying to trim down some shows

- if you look up Fullmetal Alchemist, there are like a ton of movies associated with it. I only want the main series show, so will try to remove these shows

In [17]:
df_anime[df_anime['title'].str.contains("Gintama")]

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
11,11,34096,Gintama.,After joining the resistance against the bakuf...,Action Comedy Historical Parody Samurai Sci-Fi...,"Jan 9, 2017 to Mar 27, 2017",12.0,139309,800,15.0,8.94,"joining resistance bakufu , gintoki gang hidin...",0.048907
12,12,15335,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,When Gintoki apprehends a movie pirate at a pr...,Action Sci-Fi Comedy Historical Parody Samurai...,"Jul 6, 2013",1.0,135232,833,14.0,8.95,"gintoki apprehends movie pirate premiere , che...",0.050927
13,13,918,Gintama,"The Amanto, aliens from outer space, have inva...",Action Comedy Historical Parody Samurai Sci-Fi...,"Apr 4, 2006 to Mar 25, 2010",201.0,579150,97,13.0,8.97,"amanto , alien outer space , invaded earth tak...",0.005876
15,15,15417,Gintama': Enchousen,"While Gintoki Sakata was away, the Yorozuya fo...",Action Comedy Historical Parody Samurai Sci-Fi...,"Oct 4, 2012 to Mar 28, 2013",13.0,166446,667,11.0,8.98,"gintoki sakata away , yorozuya found new leade...",0.040766
249,249,10643,Gintama: Dai Hanseikai,Some of the characters get together and talk a...,Action Comedy Parody Samurai,"Mar 25, 2010",1.0,23908,2922,560.0,8.01,character get together talk `` regret '' 4 yea...,0.178797
306,306,2951,Gintama: Nanigoto mo Saiyo ga Kanjin nano de T...,This special is a set of short comedy stories ...,Action Sci-Fi Comedy Historical Parody Samurai...,"Sep 24, 2005",1.0,39842,2195,495.0,8.06,special set short comedy story involving ginto...,0.134296
398,398,25313,Gintama': Futon ni Haitte kara Buki Nokoshi ni...,Bundled with limited edition of the 58th Ginta...,Action Sci-Fi Comedy Historical Parody Samurai...,"Apr 3, 2015",1.0,24481,2890,405.0,8.14,bundled limited edition 58th gintama manga vol...,0.176838
468,468,9735,Gintama: Shinyaku Benizakura-hen,"According to Tsutaya Online, the DVD of Gintam...",Action Comedy Historical Parody Samurai Sci-Fi...,"Dec 15, 2010",1.0,25613,2807,334.0,8.18,"according tsutaya online , dvd gintama : shiny...",0.171757
567,567,6945,Gintama: Shiroyasha Koutan,"According to Aniplex, a 10min short anime of G...",Action Sci-Fi Comedy Historical Parody,"Sep 21, 2008",1.0,44344,2051,229.0,8.30,"according aniplex , 10min short anime gintama ...",0.125482
579,579,21899,Gintama: Yorinuki Gintama-san on Theater 2D,"Demonic Vice-Commander of the Shinsengumi, Tou...",Action Sci-Fi Comedy Historical Parody Samurai...,"Aug 26, 2012",2.0,24881,2867,215.0,8.32,"demonic vice-commander shinsengumi , toushirou...",0.175430


In [9]:
df_anime[df_anime['episodes'] == 3]

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
37,37,1689,Byousoku 5 Centimeter,"Takaki Toono and Akari Shinohara, two very clo...",Slice_of_Life Drama Romance,"Mar 3, 2007",3.0,554071,108,781.0,7.86,"takaki toono akari shinohara , two close frien...",0.006550
64,64,18397,Shingeki no Kyojin OVA,"Bundled with the 12th, 13th, and 14th limited-...",Action Super Power Drama Fantasy Shounen,"Dec 9, 2013 to Aug 8, 2014",3.0,222243,480,751.0,7.88,"bundled 12th , 13th , 14th limited-edition man...",0.029320
88,88,4872,Goku Sayonara Zetsubou Sensei,Sayonara Zetsubou Sensei OVA.,Comedy Parody School Shounen,"Oct 17, 2008 to Feb 17, 2009",3.0,51621,1858,726.0,7.89,sayonara zetsubou sensei ovum .,0.113668
142,142,9088,Saiyuuki Gaiden,An OVA adaptation of the climactic part of Sa...,Action Adventure Drama Fantasy,"Mar 25, 2011 to Nov 25, 2011",3.0,9958,4352,672.0,7.93,ovum adaptation climactic part saiyuuki gaiden .,0.266328
155,155,2143,Saiyuuki Reload: Burial,Taking place in the past it offers a view at t...,Action Adventure Drama Fantasy Shoujo,"Apr 28, 2007 to Mar 28, 2008",3.0,10169,4309,659.0,7.94,taking place past offer view saiyuuki quartet ...,0.263696
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15947,18722,34649,Soukyuu no Fafner: Dead Aggressor - The Beyond,blank,Action Military Sci-Fi Drama Mecha,"May 17, 2019",3.0,4598,5965,2888.0,7.22,blank,0.365061
15973,18748,1352,Cyber City Oedo 808,It is the year 2808. Three convicts are recrui...,Action Adventure Police Sci-Fi,"Jun 21, 1990 to Oct 4, 1991",3.0,16066,3563,2860.0,7.22,year 2808. three convict recruited member cybe...,0.218033
16102,18879,1215,Mobile Suit Gundam SEED C.E.73: Stargazer,"In C.E. 73, tensions between Naturals and Coor...",Space Mecha Military Sci-Fi,"Jul 14, 2006 to Sep 29, 2006",3.0,18516,3319,2971.0,7.20,"c.e . 73 , tension natural coordinator continu...",0.203097
16106,18883,4195,Kodomo no Jikan: Ni Gakki,The second season of Kodomo no Jikan.,Comedy Ecchi School,"Feb 20, 2009 to Jul 24, 2009",3.0,26636,2741,2964.0,7.20,second season kodomo jikan .,0.167717


In [10]:
df_anime[(df_anime['episodes'] > 0) & (df_anime['episodes'] < 5)]

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
4,4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,Action Mystery Supernatural Vampire,"Jan 6, 2017",1.0,214621,502,22.0,8.83,helping revive legendary vampire kiss-shot ace...,0.030667
6,6,199,Sen to Chihiro no Kamikakushi,"Stubborn, spoiled, and naïve, 10-year-old Chih...",Adventure Supernatural Drama,"Jul 20, 2001",1.0,913212,40,20.0,8.90,"stubborn , spoiled , naïve , 10-year-old chihi...",0.002387
12,12,15335,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,When Gintoki apprehends a movie pirate at a pr...,Action Sci-Fi Comedy Historical Parody Samurai...,"Jul 6, 2013",1.0,135232,833,14.0,8.95,"gintoki apprehends movie pirate premiere , che...",0.050927
17,17,16664,Kaguya-hime no Monogatari,"Deep in the countryside, a man named Okina wor...",Fantasy Historical,"Nov 23, 2013",1.0,99563,1106,250.0,8.28,"deep countryside , man named okina work bamboo...",0.067638
18,18,4672,Koukaku Kidoutai 2.0,Mamoru Oshii's first Ghost in the Shell cybers...,Action Sci-Fi Police Psychological Mecha Seinen,"Jul 12, 2008",1.0,72331,1457,500.0,8.06,mamoru oshii 's first ghost shell cyberspace f...,0.089123
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16206,18996,34318,Mekakucity V's,Mekakucity V's compiles more songs of the Ka...,Music Romance Super Power Supernatural,"Jan 1, 2014",4.0,3802,6428,1977.0,7.43,mekakucity v 's compiles song kagerou project ...,0.393401
16208,18999,36043,Net-juu no Susume Special,An unaired 11th episode included with the Blu-...,Game Comedy Romance,"Dec 8, 2017",1.0,62448,1630,1802.0,7.48,unaired 11th episode included blu-ray box set ...,0.099712
16209,19002,10075,Naruto x UT,All-new animation offered throughout UNIQLO cl...,Action Comedy Super Power Martial Arts Shounen,"Jan 1, 2011",1.0,34155,2382,1728.0,7.50,all-new animation offered throughout uniqlo cl...,0.145743
16212,19005,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",Action,"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",0.131787


In [11]:
df_anime[df_anime['episodes'] == 1].sort_values(by = 'popularity', ascending=True).head(10)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
771,771,32281,Kimi no Na wa.,"Mitsuha Miyamizu, a high school girl, yearns t...",Romance Supernatural School Drama,"Aug 26, 2016",1.0,1139878,15,4.0,9.09,"mitsuha miyamizu , high school girl , yearns l...",0.000857
6,6,199,Sen to Chihiro no Kamikakushi,"Stubborn, spoiled, and naïve, 10-year-old Chih...",Adventure Supernatural Drama,"Jul 20, 2001",1.0,913212,40,20.0,8.90,"stubborn , spoiled , naïve , 10-year-old chihi...",0.002387
765,765,28851,Koe no Katachi,"As a wild youth, elementary school student Sho...",Drama School Shounen,"Sep 17, 2016",1.0,842277,53,10.0,9.01,"wild youth , elementary school student shouya ...",0.003183
763,763,164,Mononoke Hime,When an Emishi village is attacked by a fierce...,Action Adventure Fantasy,"Jul 12, 1997",1.0,656961,77,30.0,8.77,"emishi village attacked fierce demon boar , yo...",0.004652
749,749,431,Howl no Ugoku Shiro,"That jumbled piece of architecture, that cacop...",Adventure Drama Fantasy Romance,"Nov 20, 2004",1.0,615722,88,41.0,8.71,"jumbled piece architecture , cacophony hissing...",0.005325
650,650,523,Tonari no Totoro,"In 1950s Japan, Tatsuo Kusakabe relocates hims...",Adventure Comedy Supernatural,"Apr 16, 1988",1.0,528128,120,143.0,8.42,"1950s japan , tatsuo kusakabe relocates two da...",0.007284
543,543,2236,Toki wo Kakeru Shoujo,Makoto Konno is in her last year of high schoo...,Adventure Drama Romance Sci-Fi,"Jul 15, 2006",1.0,509087,130,255.0,8.28,"makoto konno last year high school , hard time...",0.007896
747,747,12355,Ookami Kodomo no Ame to Yuki,"Hana, a hard-working college student, falls in...",Fantasy Slice_of_Life,"Jul 21, 2012",1.0,472563,150,43.0,8.70,"hana , hard-working college student , fall lov...",0.009120
476,476,16782,Kotonoha no Niwa,"On a rainy morning in Tokyo, Takao Akizuki, an...",Slice_of_Life Psychological Drama Romance,"May 31, 2013",1.0,420047,190,326.0,8.19,"rainy morning tokyo , takao akizuki , aspiring...",0.011569
688,688,10408,Hotarubi no Mori e,"Intrigued by the tale of a mountain god, six-y...",Drama Romance Shoujo Supernatural,"Sep 17, 2011",1.0,415415,193,104.0,8.48,"intrigued tale mountain god , six-year-old hot...",0.011752


## What to do:
- after looking at the df's, i think anything 2 eps or less should be taken out. These are not whole series, rather they are shorts, movies, or OVA's. Not something i'd recommend to hook someone into a show
- Some movies in the top 1000 I do believe are worthy so I will keep those. 

In [12]:
df_anime_nomovie = df_anime[(df_anime['popularity'] <= 1000) | (df_anime['popularity'] > 1000) & (df_anime['episodes'] > 2)]

df_anime_nomovie

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,Comedy Sports Drama School Shounen,"Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",0.008570
1,1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,Drama Music Romance School Shounen,"Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,"music accompanies path human metronome , prodi...",0.001653
2,2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,Sci-Fi Adventure Mystery Drama Fantasy,"Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,"abyss—a gaping chasm stretching depth earth , ...",0.005937
3,3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",Action Military Adventure Comedy Drama Magic F...,"Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,"`` order something obtained , something equal ...",0.000184
4,4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,Action Mystery Supernatural Vampire,"Jan 6, 2017",1.0,214621,502,22.0,8.83,helping revive legendary vampire kiss-shot ace...,0.030667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16206,18996,34318,Mekakucity V's,Mekakucity V's compiles more songs of the Ka...,Music Romance Super Power Supernatural,"Jan 1, 2014",4.0,3802,6428,1977.0,7.43,mekakucity v 's compiles song kagerou project ...,0.393401
16207,18998,8676,Amagami SS,"Two years ago, Junichi Tachibana had a date on...",Slice_of_Life Comedy Romance School,"Jul 2, 2010 to Dec 24, 2010",25.0,240377,417,1885.0,7.45,"two year ago , junichi tachibana date christma...",0.025464
16210,19003,35828,Miira no Kaikata,High school student Sora Kashiwagi is accustom...,Slice_of_Life Comedy Supernatural,"Jan 12, 2018 to Mar 30, 2018",12.0,61459,1648,1727.0,7.50,high school student sora kashiwagi accustomed ...,0.100814
16211,19004,10378,Shinryaku!? Ika Musume,"After regaining her squid-like abilities, Ika ...",Slice_of_Life Comedy Shounen,"Sep 27, 2011 to Dec 25, 2011",12.0,67422,1547,1548.0,7.56,"regaining squid-like ability , ika musume plan...",0.094632


In [13]:
df_anime[(df_anime['popularity'] > 1000) & (df_anime['episodes'] > 2)]

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
20,20,7588,Saraiya Goyou,Masterless samurai Akitsu Masanosuke is a skil...,Mystery Historical Drama Samurai Seinen,"Apr 16, 2010 to Jul 2, 2010",12.0,61146,1650,800.0,7.86,masterless samurai akitsu masanosuke skilled l...,0.100937
21,21,1253,Saint Seiya: Meiou Hades Meikai-hen,Seiya and the rest of the Bronze and Gold Sain...,Action Adventure Demons Fantasy Sci-Fi Shounen...,"Dec 17, 2005 to Feb 6, 2007",12.0,33453,2409,799.0,7.86,seiya rest bronze gold saint survived sanctuar...,0.147395
23,23,1425,Lupin III: Part II,Lupin III chronicles the adventures of Arsene ...,Action Adventure Mystery Comedy Seinen,"Oct 3, 1977 to Oct 6, 1980",155.0,19405,3237,796.0,7.86,lupin iii chronicle adventure arsene lupin iii...,0.198078
25,25,22673,Kuroko no Basket 2nd Season NG-shuu,"Animated bloopers, based on the extra section ...",Comedy Sports,"Jan 29, 2014 to Sep 24, 2014",9.0,38046,2263,795.0,7.86,"animated blooper , based extra section origina...",0.138459
32,32,36369,Ginga Eiyuu Densetsu: Die Neue These - Seiran 1,The Free Planets Alliance's invasion of the Ga...,Action Drama Military Sci-Fi Space,"Sep 27, 2019",4.0,11817,4041,787.0,7.86,free planet alliance 's invasion galactic empi...,0.247291
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16199,18983,27829,Heavy Object,"In the distant future, the nature of war has c...",Action Military Sci-Fi Mecha,"Oct 3, 2015 to Mar 26, 2016",24.0,111872,1008,2367.0,7.34,"distant future , nature war changed . `` objec...",0.061639
16201,18985,24625,Gundam Build Fighters Try,The story of Gundam Build Fighters Try is set ...,Action Sci-Fi Mecha,"Oct 8, 2014 to Apr 1, 2015",25.0,29241,2613,2277.0,7.36,story gundam build fighter try set 7 year end ...,0.159882
16206,18996,34318,Mekakucity V's,Mekakucity V's compiles more songs of the Ka...,Music Romance Super Power Supernatural,"Jan 1, 2014",4.0,3802,6428,1977.0,7.43,mekakucity v 's compiles song kagerou project ...,0.393401
16210,19003,35828,Miira no Kaikata,High school student Sora Kashiwagi is accustom...,Slice_of_Life Comedy Supernatural,"Jan 12, 2018 to Mar 30, 2018",12.0,61459,1648,1727.0,7.50,high school student sora kashiwagi accustomed ...,0.100814


In [14]:
#df_anime_nomovie.to_csv('df_anime_nomovie.csv')

In [15]:
df_anime.to_csv('df_anime_all_cleaned.csv')

In [16]:
df_anime.head(1)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,popularity_normalized
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,Comedy Sports Drama School Shounen,"Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",0.00857
